In [15]:
import csv
import pandas as pd
import os
import classify
import extract_dates
import random
import spacy
import en_core_web_sm
nlp =  en_core_web_sm.load()
import re

In [4]:
docs = pd.read_csv(os.path.join(data_path, 'text_narrowed.csv'))
test = docs[docs.title == "Ector County ISD"]

In [5]:
test

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,level,type,link,text,p_innovation
568,270,270,270,Ector County ISD,First,pdf,http://www.ectorcountyisd.org/cms/lib011/tx010...,ECISD District of Innovation Plan 1 I. Introdu...,0.999667


In [6]:
test.text.values

array(['ECISD District of Innovation Plan 1 I. Introduction House Bill 1842, passed during the 84th Legislative Session, permits Texas public school districts to become Districts of Innovation and to obtain exemption from certain provisions of the Texas Education Code. On February 21, 2017 the Ector County Independent School District’s Board of Trustees (“Board”) passed a resolution to initiate the Process of Designation as a District of Innovation in order to increase local control over District operations and to support innovation and local initiatives to improve educational outcomes for the benefit of students and the community. On February 28, 2017, the Board appointed an eighteen member District of Innovation Committee (“Committee”) comprised of district leaders, teachers, parents, and community members representing a variety of roles and responsibilities. The Committee met on March 2, 6, and 10, 2017, to discuss and draft this Local Innovation Plan (“Plan”). Based on direction pr

In [8]:
# TODO change number of tokens to sentence
def get_phrase_list(text, n_tokens_before=8, n_tokens_after=6):
    nlp = en_core_web_sm.load()
    doc = nlp(text)
    phrases = []
    for token in doc:
        bad_match = re.findall('\.', token.text)
        if not bad_match and len(token) == 4:
            date = re.findall('\d\d\d\d', token.text)
            if date:
                i = token.i
                start = i - n_tokens_before
                if start < 0:
                    start = 0
                end = i + (n_tokens_after + 1)
                phrase = doc[start:end]
                phrases.append(phrase)
                phrases = [str(item) for item in phrases]
    return phrases

In [33]:
for text in test.text:
    doc = nlp(text)
    phrases = []
    for token in doc:
        bad_match = re.findall('\.', token.text)
        if not bad_match and len(token) == 4:
            date = re.findall('\d\d\d\d', token.text)
            if date:
                i = token.i
                start = i - n_tokens_before
                if start < 0:
                    start = 0
                while doc[start].ent_iob_ == 'I':
                    start = start - 1
                end = i + (n_tokens_after + 1)
                while doc[end].ent_iob_ == 'I':
                    end = end + 1
                phrase = doc[start:end]
                phrases.append(phrase)
                phrases = [str(item) for item in phrases]

In [34]:
phrases

['of Innovation Plan 1 I. Introduction House Bill 1842, passed during the 84th Legislative Session',
 'the Texas Education Code. On February 21, 2017 the Ector County Independent School District’s Board of Trustees (“Board”',
 'and the community. On February 28, 2017, the Board appointed an eighteen',
 'March 2, 6, and 10, 2017, to discuss and draft this Local Innovation Plan',
 'term of the Plan will begin with the 2017-2018 school year and terminate',
 'the Plan will begin with the 2017-2018 school year and terminate at the',
 '2017-2018 school year and terminate at the end of the 2021-2022 school year, to',
 'terminate at the end of the 2021-2022 school year, to include anything',
 'appointment of DOI members* February 14, 2016 February Board Meeting \uf0b7 Board votes',
 'as a District of Innovation February 21, 2016 February \uf0b7 Board holds public hearing',
 'final Local Innovation Plan on April 11, 2017* February 28, 2017 March DOI',
 'April 11, 2017* February 28, 2017 March D

In [37]:
small = 'House Bill 1842, passed during the 84th Legislative Session, permits Texas public school districts to become Districts of Innovation and to obtain exemption from certain provisions of the Texas Education Code.'
small_doc = nlp(small)
for token in small_doc:
    print(token, ':', token.left_edge)

House : House
Bill : House
1842 : 1842
, : ,
passed : House
during : during
the : the
84th : 84th
Legislative : Legislative
Session : the
, : ,
permits : House
Texas : Texas
public : public
school : public
districts : Texas
to : to
become : to
Districts : Districts
of : of
Innovation : Innovation
and : and
to : to
obtain : to
exemption : exemption
from : from
certain : certain
provisions : certain
of : of
the : the
Texas : Texas
Education : Education
Code : the
. : .


In [38]:
n_tokens_before = 8
n_tokens_after = 6

In [47]:
doc = nlp('Sulphur Bluff ISD District Of Innovation Plan 2017-2018 School Year To 2021-2022 School Year')
phrases = []
for token in doc:
    bad_match = re.findall('\.', token.text)
    if not bad_match and len(token) == 4:
        date = re.findall('\d\d\d\d', token.text)
        if date:
            i = token.i
            start = i - n_tokens_before
            if start < 0:
                start = 0
            while doc[start].ent_iob_ == 'I':
                start = start - 1
            end = i + n_tokens_after
            if end >= len(doc):
                end = len(doc) - 1
            while end < len(doc) and doc[end].ent_iob_ == 'I':
                end = end + 1
            phrase = doc[start:end]
            phrases.append(phrase)
            phrases = [str(item) for item in phrases]

In [49]:
text = 'and the community. On February 28, 2017, the Board appointed an eighteen'

In [73]:
month_shape_patterns

[[{'ORTH': 'January'}],
 [{'ORTH': ' February '}],
 [{'ORTH': 'March'}],
 [{'ORTH': 'April'}],
 [{'ORTH': 'May'}],
 [{'ORTH': 'June'}],
 [{'ORTH': 'July'}],
 [{'ORTH': 'August'}],
 [{'ORTH': 'September'}],
 [{'ORTH': 'October'}],
 [{'ORTH': 'November'}],
 [{'ORTH': 'December'}]]

In [100]:
from spacy.matcher import Matcher
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
month_shape_patterns = [[{'ORTH':month}] for month in months] 

matcher = Matcher(nlp.vocab)
matcher.add("month", None, *month_shape_patterns)

def get_matches(string):
    doc = nlp(string)
    matches = matcher(doc)
    return list(set([doc[i[1]:i[2]][0] for i in matches]))


In [101]:
get_matches(text)

[February]

In [102]:
text

'and the community. On February 28, 2017, the Board appointed an eighteen'

In [69]:
doc = nlp(text)
matches = matcher(doc)
list(set([doc[i[1]:i[2]][0] for i in matches]))

[]

In [64]:
matches

[]

In [6]:
from start import data_path
import os
from pathlib import Path
import pandas as pd

In [4]:
x

1

In [8]:
data_path = os.path.join(data_path)
data_path

'/Users/kylieleblancKylie/dofis/code/exemptions/../../data/plans'

In [10]:
filename = Path("source_data/text_files/raw_data.txt")

In [11]:
filename

PosixPath('source_data/text_files/raw_data.txt')

In [8]:
docs = pd.read_csv(os.path.join(data_path, 'text_narrowed.csv'))

In [9]:
docs

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,level,type,link,text,p_innovation
0,672,672,672,Lake Travis ISD,Second,pdf,https://www.ltisdschools.org//cms/lib/Tx018000...,"Effective Dec. 13, 2016 – Dec. 13, 2021 2 What...",0.999955
1,1243,1243,1243,Zephyr ISD,Second,docx,http://zephyrisd.net/wp-content/uploads/2014/0...,Zephyr ISD District of Innovation Plan Introdu...,0.999922
2,315,315,315,Zavalla ISD,First,pdf,http://www.zavallaisd.org/Uploads/269/misc/f28...,Zavalla ISD District of Innovation Plan (HB 18...,0.999619
3,3309,3309,3309,Zapata County ISD,Second,pdf,https://1.cdn.edl.io/VZ6fi1M6pn8Qk8tS3GetGewCD...,Microsoft Word - Document1 ZAPATA COUNTY INDEP...,0.452118
4,1196,1196,1196,Yorktown ISD,Second,pdf,http://www.yisd.org/userfiles/57/my%20files/fi...,Yorktown Independent School District Final Dis...,0.999877
5,334,334,334,Yoakum ISD,First,pdf,http://www.yoakumisd.net/cms/lib3/TX01001553/C...,DISTRICT OF INNOVATION DISTRICT OF INNOVATION ...,0.998961
6,3667,3667,324,Wylie ISD (221912),html,html,http://www.wyliebulldogs.org/cms/One.aspx?port...,District of Innovation - Wylie Independent Sch...,0.012857
7,276,276,276,Wylie ISD (043914),First,pdf,http://www.wylieisd.net/cms/lib09/TX01918453/C...,Wylie ISD INNOVATION PLAN INTRODUCTION House B...,0.999891
8,1542,1542,1542,Wortham ISD,Second,docx,https://s3.amazonaws.com/scschoolfiles/888/wis...,W Wortham ISD District of Innovation Plan 2017...,0.997949
9,350,350,350,Woodville ISD,First,pdf,https://core-docs.s3.amazonaws.com/documents/a...,No text,0.001259
